# Factory Simulation with SimPy
## Scenario
A factory gets **raw material**. The raw material is processed/assembled into pieces by **Piece Assembly Machines**. Then, the pieces are assembled into models by the **Model Assembly Machines**. From time to time, the machines break.

When a machine breaks, the factory calls a repair center, which has other callers. When the factory's request reaches an operator, they decide if intervention is necessary. If no intervention is needed, the factory workers can repair the machine by themselves. If intervention is necessary, a repairman is called to the premises, who then repairs the machine.

### Math Reminders - Exponential & Lognormal Distributions
They can be found in `pharmacy_sim.ipynb`. 

### Math Reminder - Normal Distribution
Also called the **Gaussian Distribution** is one of the most commonly used distributions in statistics.

Its PDF is:

$$
f(x) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp(-\frac{(x-\mu)^2}{2\sigma^2})
$$

In [ ]:
import numpy as np
import simpy
import pandas as pd
import itertools
from distributions import (
    Exponential,
    Lognormal,
    Normal,
    Bernoulli
)

In [ ]:
# Default Parameters

N_OPERATORS = 5
N_REPAIRMEN = 3
CALL_RATE = 3
CALL_SCALE = 1 / CALL_RATE
TRAVEL_SIGMA = 0.04
TRAVEL_MU = 2.7
CALL_LENGTH_SIGMA = 0.1
CALL_LENGTH_MU = 1.7
REPAIR_TIME_MEAN = 8.5
REPAIR_TIME_VAR = 0.7
INTERVENTION_CHANCE = 0.2  
N_STREAMS = 4
DEFAULT_RND_SET = 42
TRACE = False
COLLECTION_TIME = 23 * 60
WARMUP_TIME = 1 * 60

In [ ]:
class Experiment:

    def __init__(
        self,
        random_nr_set = DEFAULT_RND_SET,
        n_streams = N_STREAMS,
        n_operators = N_OPERATORS,
        n_repairmen = N_REPAIRMEN,
        call_scale = CALL_SCALE,
        call_length_sigma = CALL_LENGTH_SIGMA,
        call_length_mu = CALL_LENGTH_MU,
        travel_sigma = TRAVEL_SIGMA,
        travel_mu = TRAVEL_MU,
        repair_time_mean = REPAIR_TIME_MEAN,
        repair_time_var = REPAIR_TIME_VAR,
        intervention_chance = INTERVENTION_CHANCE
    ):
        self.random_nr_set = random_nr_set
        self.n_streams = n_streams
        self.n_operators = n_operators
        self.n_repairmen = n_repairmen
        self.call_scale = call_scale
        self.call_length_sigma = call_length_sigma
        self.call_length_mu = call_length_mu
        self.travel_sigma = travel_sigma
        self.travel_mu = travel_mu
        self.repair_time_mean = repair_time_mean
        self.repair_time_var = repair_time_var
        self.intervention_chance = intervention_chance

        self.operators = None
        self.repairmen = None

        self.init_results_vars()
        self.init_sampling()


    def set_random_nr_set(self, random_nr_set):
        self.random_nr_set = random_nr_set
        self.init_sampling()

    
    def init_sampling(self):
        seed_sequence = np.random.SeedSequence(self.random_nr_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        self.arrival_dist = Exponential(
            self.call_scale, 
            self.seeds[0]
        )
        self.call_dist = Lognormal(
            self.call_length_mu,
            self.call_length_sigma,
            self.seeds[1]
        )
        self.travel_dist = Lognormal(
            self.travel_mu,
            self.travel_sigma,
            self.seeds[2]
        )
        self.repair_dist = Normal(
            self.repair_time_mean,
            self.repair_time_var,
            self.seeds[3]
        )
        self.intervention_dist = Bernoulli(
            self.intervention_chance,
            self.seeds[4]
        )

    def init_results_vars(self):
        self.results = {}
        self.results["call_waiting times"] = []
        self.results["total_call_duration"] = 0.0
        self.results["repairman_dispatch_waiting_times"] = []
        self.results["total_repairman_usage"] = 0.0

In [ ]:
def trace(msg):
    if TRACE:
        print(msg)

In [ ]:
def intervention(id, env, args):
    trace(f"Caller #{id} waiting for repairman")
    start_dispatch_wait = env.now
    with args.repairmen.request() as req:
        yield req
        repairman_dispatch_time = env.now - start_dispatch_wait
        args.results["repairman_dispatch_waiting_times"].append(repairman_dispatch_time)
        travel_duration = args.travel_dist.sample()
        trace(f"Repairman dispatched for caller #{id} at {env.now:.2f}")
        yield env.timeout(travel_duration)
        trace(f"Repairman arrived at caller #{id} at {env.now:.2f}")
        repair_duration = args.repair_dist.sample()
        yield env.timeout(repair_duration)
        trace(f"Repairman finished reairing stuff for caller #{id} at {env.now}")
        args.results["total_repairman_usage"] += (travel_duration + repair_duration)

In [ ]:
def call_service(id, env, args):
    start_wait = env.now
    
    with args.operators.request() as req:
        yield req
        waiting_time = env.now - start_wait
        args.results["call_waiting_times"].append(waiting_time)
        trace(f"An operator answered call #{id} at {env.now:.2f}")
        call_duration = args.call_dist.sample()
        yield env.timeout(call_duration)
        args.results["total_call_duration"] += call_duration
        trace(f"Call #{id} ended at {env.now:.2f}. Waiting time was {waiting_time:.2f}")

        need_intervention = args.intervention_dist.sample()
        if need_intervention:
            env.process(intervention(id, env, args))

In [ ]:
def generator(env, args):
    for caller_count in itertools.count(start=1):
        inter_call_time = args.arrival_dist.sample()
        yield env.timeout(inter_call_time)
        trace(f"A call at {env.now:.2f}")
        env.process(call_service(caller_count, env, args))